In [1]:
# Rio de Janeiro, 24 de Setembro de 2021

# Outras referencias (non patent references)
# O que seria relevante de extrair? 
# - Contagem de referencias
# - Se a referencia e' um paper cientifico
# Este script deve extrair a info relevante para se incluido no modelo

# Quatro campos: uuid | patent_id | text | sequence


import dask.dataframe as dd
import glob
from dask.delayed import delayed
import pandas as pd
import re


# Other references (non patent citation)

def clean_field(df):
    cleaning_patent=lambda x:re.sub('([^a-zA-Z0-9]+)', "", x)
    df.dropna(inplace=True)
    df['text']=df['text'].apply(cleaning_patent)
    return df


In [2]:
def run_processes(dfs):
    for i, df in enumerate(dfs):
        myTypes={'uuid':str, 'patent_id':str, 'text':str, 'sequence':str}
        df=dd.from_delayed(df, meta=myTypes)
        df=delayed(clean_field)(df)
        dst='data/clean_otherref_'+str(i)+'.parquet.gz'       
        df.set_index('patent_id').compute().to_parquet(dst, compression='gzip')

In [3]:
file_list=glob.glob("parquet/otherref*")
dfs = [delayed(pd.read_parquet)(f) for f in file_list]

dd.compute(run_processes(dfs))

(None,)